In [1]:
import pandas as pd
import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import ast
import csv
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
FOLDER_PATH = "/content/drive/MyDrive/LLM Interpretability/"
DATASET_NAMES = ["emotion", "math", "mmlu", "programming"]
DATASET_PATHS = [f"{FOLDER_PATH}/{name}_processed.csv" for name in DATASET_NAMES]
# Load and concatenate all datasets
df = pd.concat([pd.read_csv(path, encoding="utf-8-sig") for path in DATASET_PATHS], ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/LLM Interpretability//programming_processed.csv'

In [ ]:
df.dropna(inplace=True)    # for analysis and vis keep all
df.head()

In [ ]:
counts = df['type'].value_counts()
print(counts)

In [ ]:
df.describe()

In [ ]:
df.count()

In [ ]:
# create freq of vectors [0, 0, 0, .., 0]
VECTOR_SIZE_SAE = 16384
def to_vector(specialized_df, col_name, binary_vector=False):
    """
    input:
        specialized_df: pandas df
        col_name: col name with its layer and type
    output:
        vector of size 16500
            max neuron 16383
    """
    shared_neurons = [0] * VECTOR_SIZE_SAE
    for row_list_str in specialized_df[col_name]:      # TODO: optimize with counter
        # skip missing entries
        if pd.isna(row_list_str):
            continue
        neuron_list = ast.literal_eval(row_list_str)
        for neuron in neuron_list:
            if binary_vector:
                shared_neurons[neuron] = 1
            else:
                shared_neurons[neuron] += 1

    return shared_neurons

# to_vector(df, "gemma-scope-2b-pt-res-canonical-layer0-token_feature_ids")
# first_list = ast.literal_eval(df["gemma-scope-2b-pt-res-canonical-layer0-token_feature_ids"].iloc[0] )  # now a Python list of ints

In [ ]:
DATASET_TYPES = {
    "empathetic_dialogue": df.loc[df["type"] == "empathetic_dialogue", "category"].unique(),
    "math":                df.loc[df["type"] == "math",                "category"].unique(),
    "mmlu":                df.loc[df["type"] == "mmlu",                "category"].unique(),
    "programming":         df.loc[df["type"] == "programming",         "category"].unique(),
}
print(DATASET_TYPES)


# DATASET_TYPES = [
#                 "empathetic_dialogue",
#                 "math",
#                 "mmlu",
#                 "programming"
#                 ]

In [ ]:
# WITHIN CLASS

import itertools

TYPE = ["res", "mlp", "att"]
LAYERS = range(26)

results = []

for typ in TYPE:
    for layer in LAYERS:
        col = f"gemma-scope-2b-pt-{typ}-canonical-layer_{layer}/width_16k/canonical-token_feature_ids"

        # 2. For each dataset type, loop over all unique (category1, category2) pairs
        for dataset_type, categories in DATASET_TYPES.items():
            for class1, class2 in itertools.combinations(categories, 2):
                # filter down to just those two classes within this dataset type
                df1 = df[(df["type"] == dataset_type) & (df["category"] == class1)]
                df2 = df[(df["type"] == dataset_type) & (df["category"] == class2)]

                vec1 = to_vector(df1, col, False)
                vec2 = to_vector(df2, col, False)

                common_count, activation_pct = percentage_activation(vec1, vec2)

                results.append({
                    "Type": typ,
                    "Layer": layer,
                    "DatasetType": dataset_type,
                    "Class1": class1,
                    "Class2": class2,
                    "ActivationPercentage": activation_pct
                })

# at the end you can wrap results in a DataFrame if you like:
# import pandas as pd
# results_df = pd.DataFrame(results)
# print(results_df)
